# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

In [ ]:
%pip install -r requirements.txt


In [ ]:
# Imports
import os
import pandas as pd
import pyterrier as pt
import transformers
import torch
import nltk
import spacy
import shutil
#import matplotlib
import numpy as np
from collections import defaultdict

In [ ]:
import os
os.makedirs("data", exist_ok=True)

!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o data/test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o data/train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o data/validation.json

import json

files = ["data/train.json", "data/validation.json", "data/test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [ ]:
import json
import os

inputs = ["data/train.json", "data/validation.json", "data/test.json"]
output = "data/document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

## You should check that the collection you have matches that of the paper!

In [ ]:
import pandas as pd
for path in inputs:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        df_check = pd.read_json(path)
        print(f'Shape of {path}: {df_check.shape}')

The dimensions match the ones of the paper at https://github.com/DataScienceUIBK/ChroniclingAmericaQA

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [ ]:
import json
import re
import unicodedata
import string

input_file = "data/test.json"
output_file = "data/test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

# Create the Qrels for the test set

In [ ]:
input_file = "data/test.json"
qrels_file = "data/test_qrels.json"
answers_file = "data/test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

# Retrieval

### Extract data from json files

In [ ]:
input_files = ['data/document_collection.json', 'data/test.json', 'data/test_qrels.json', 'data/test_queries.json', 'data/test_query_answers.json', 'data/train.json', 'data/validation.json']

dataframes = {}
for input_file in input_files:
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        dataframes[input_file] = pd.read_json(input_file)

Let's visualize data and analyze them

In [ ]:
dataframes['data/document_collection.json']

In [ ]:
dataframes['data/train.json']

**NOTE: in `data/document_collection.json` the rows are already deduplicated**

### _Preprocessing_

#### **Linguistic Processing**

##### Normalization
We lowercase everything and remove all special characters/tags

--> 1st step normalization

In [ ]:
def normalize_text1(text):
    if not isinstance(text, str):
        return text
    text = unicodedata.normalize('NFKC', text)
    #text = text.lower()
    text = re.sub(r'<[^>]+>', ' ', text) # HTML
    # text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip() # multiple white spaces
    return text

# in caso togliessimo la NER vanno tolti i commenti nella funzione qui sopra

docColl = dataframes['data/document_collection.json']
docColl_contNorm1 = docColl['context'].apply(normalize_text1)
docColl_ocrNorm1 = docColl['raw_ocr'].apply(normalize_text1)
docColl_Norm1 = docColl.copy()

In [ ]:
docColl_Norm1['context'] = docColl_contNorm1
docColl_Norm1['raw_ocr'] = docColl_ocrNorm1
docColl_Norm1.head(25)

In [ ]:
docColl['context'].compare(docColl_Norm1['context'])

In [ ]:
print(docColl['context'].iloc[2])
print(docColl_Norm1['context'].iloc[2])

##### NER
We want to identify named-entities before lemmatizing the text, so that we do not lose any entity by "shrinking" words to their base forms.

In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, pipeline
from tqdm import tqdm

# Settings per far runnare su gpu (se possibile)
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
device = "mps" if torch.backends.mps.is_available() else "cpu"

MODEL_NAME = "impresso-project/ner-stacked-bert-multilingual-light"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ner_pipeline = pipeline(
    model=MODEL_NAME,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device=device)

def run_impresso_ner(text_series):
    results = []
    for text in tqdm(text_series): # tqdm per vedere i progressi nelle ore di run
        text_str = str(text)
        if not text_str.strip(): # per testi vuoti
            results.append([])
            continue

        words = text_str.split()

        try:
            entities = ner_pipeline(text_str, tokens=words)
            results.append(entities)
        except Exception as e:
            print(f"Errore su un documento: {e}")
            results.append([]) # per non farlo bloccare se ha un errore
    return results

OUTPUT_FILE = "data/ner_results_cache.parquet"
if os.path.exists(OUTPUT_FILE):
    cached_data = pd.read_parquet(OUTPUT_FILE)

    docColl_Norm1['ner_entities_context'] = cached_data['ner_entities_context']
    docColl_Norm1['ner_entities_ocr'] = cached_data['ner_entities_ocr']

else:
    # context
    docColl_Norm1['ner_entities_context'] = run_impresso_ner(docColl_Norm1['context'])
    # OCR
    docColl_Norm1['ner_entities_ocr'] = run_impresso_ner(docColl_Norm1['raw_ocr'])
    # salvataggio su file esterno
    docColl_Norm1[['ner_entities_context', 'ner_entities_ocr']].to_parquet(OUTPUT_FILE)

In [ ]:
docColl_Norm1

In [ ]:
docColl_ner = docColl_Norm1.copy()
docColl_ner[['context', 'raw_ocr', 'ner_entities_context', 'ner_entities_ocr']]

--> 2nd step normalization

In [ ]:
def normalize_text2(text):
    if not isinstance(text, str):
        return text
    text = text.lower() # lowercase
    text = re.sub(r'[^a-z0-9\s]', '', text) # punctuations
    text = re.sub(r'\s+', ' ', text).strip() # white spaces again
    return text

# da testare così,
# se va: cambiare anche normalize_text1
# se non va: scrivere questo apply(normalize_text2) diviso tra context e raw_ocr e poi riunire tutto su un dataframe unico
docColl_Norm2 = docColl_ner[['context', 'raw_ocr']].apply(normalize_text2)
#docColl_ocrNorm2 = docColl_ner['raw_ocr'].apply(normalize_text2)

##### Lemmatization
Placed here to standardize semantically the sentences in the documents

In [ ]:
import spacy


OUTPUT_FILE_LEMM = "data/lemmatization_results_cache.parquet"
columns_to_process = ['context', 'raw_ocr']

try:
    # Carichiamo il modello disabilitando parser e ner per massimizzare la velocità
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
except OSError:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def run_lemmatization(df, columns):
    temp_df = pd.DataFrame(index=df.index)

    for col in columns:
        if col in df.columns:
            print(f"\n--- Elaborazione colonna: '{col}' ---")
            texts = df[col].astype(str).tolist()
            processed_texts = []

            for doc in tqdm(nlp.pipe(texts, batch_size=2000, n_process=-1), total=len(texts), desc=f"Lemmatizing {col}"):
                lemmas = [token.lemma_ for token in doc if not token.is_space]
                processed_texts.append(" ".join(lemmas))

            temp_df[f"{col}_lemma"] = processed_texts
        else:
            print(f" Avviso: Colonna '{col}' non trovata.")

    return temp_df

if os.path.exists(OUTPUT_FILE_LEMM):
    cached_lemm = pd.read_parquet(OUTPUT_FILE_LEMM)
    docColl_Norm2['context_lemma'] = cached_lemm['context_lemma']
    docColl_Norm2['raw_ocr_lemma'] = cached_lemm['raw_ocr']

else:
    # context
    lemmatized_df = run_lemmatization(docColl_Norm1[['context', 'raw_ocr']], columns_to_process)
    docColl_Norm2['context_lemma'] = lemmatized_df['context_lemma']
    docColl_Norm2['raw_ocr_lemma'] = lemmatized_df['raw_ocr_lemma']
    # salvataggio su file esterno
    docColl_Norm2[['ner_entities_context', 'raw_ocr_lemma']].to_parquet(OUTPUT_FILE_LEMM)

docColl_Lemm = docColl_Norm2

# Verifica finale
print("\nDataFrame finale: docColl_Lemm")
cols_to_show = [c for c in docColl_Lemm.columns if '_lemma' in c or c in columns_to_process]
print(docColl_Lemm[cols_to_show].head())

##### N-gram based tokenization
Important to place it after normalization, in this tokenization can be integrated a NER-aware part so that "the tokenization is also entity-guided"

In [ ]:
def ner_aware_ngram_tokenizer(row, text_col, ner_col, n=2):
    """
    Tokenizzazione N-gram aware delle entità:
    1. Unisce le entità composte (es. "new york" -> "new_york") nel testo.
    2. Genera N-grams.
    """
    text = row.get(text_col, "")
    entities = row.get(ner_col, [])

    if not isinstance(text, str) or not text.strip():
        return []

    if isinstance(entities, list) and len(entities) > 0:
        entity_texts = []

        for ent in entities:
            if ent['confidence_ner'] >= 0.5:
                entity = ent['surface']
                entity_texts.append(entity)
        
        for ent_text in sorted(entity_texts, key=len, reverse=True):
            clean_ent = ent_text.lower().strip()
            if " " in clean_ent:
                clean_ent_glued = clean_ent.replace(" ", "_")
                print(text)
                text = text.replace(clean_ent, clean_ent_glued)
                print(text)

    tokens = text.split()
    print(tokens) # check per vedere cosa esce dalla funzione

    if len(tokens) < n:
        return tokens 

    n_grams_tuples = zip(*[tokens[i:] for i in range(n)])
    
    n_grams_list = [" ".join(ngram) for ngram in n_grams_tuples]

    return n_grams_list


docColl_tok = docColl_Lemm.copy()

N_VALUE = 2

columns_map = [
    ('context_lemma', 'ner_entities_context', 'context_ngrams'),
    ('raw_ocr_lemma', 'ner_entities_ocr',     'raw_ocr_ngrams')
]

print(f" Avvio Tokenization ({N_VALUE}-grams) Entity-Aware...")

for text_col, ner_col, new_col in columns_map:
    
    if text_col in docColl_tok.columns and ner_col in docColl_tok.columns:
        print(f"\n--- Elaborazione: {text_col} + {ner_col} -> {new_col} ---")
        
        docColl_tok[new_col] = docColl_tok.apply(
            lambda row: ner_aware_ngram_tokenizer(
                row, 
                text_col=text_col, 
                ner_col=ner_col, 
                n=N_VALUE
            ), 
            axis=1
        )
        print(f"Colonna creata: {new_col}")
        
    else:
        print(f" Saltato: Colonne '{text_col}' o '{ner_col}' non trovate nel DataFrame.")




print("\nDataFrame finale: docColl_tok")
cols_to_show = ['context_lemma', 'context_ngrams', 'raw_ocr_lemma', 'raw_ocr_ngrams']
cols_existing = [c for c in cols_to_show if c in docColl_tok.columns]
print(docColl_tok[cols_existing].head())

da qui dovrebbe uscire il dataframe chiamato docColl_tok

### _Multi-field Indexing_

--> Indexing con PyTerrier usando un generator

In [ ]:
# qui assumiamo che le celle create dal NER siano oggetti di tipo dizionario
def createGenerator(df, context=True):
    # context
    if context:
        for _, row in df.iterrows():
            # togliamo lOffset and rOffset
            clean_ents = []
            for ent in row['ner_entities_context']:
                cleaned = {k: v for k, v in ent.items() if k not in ['lOffset', 'rOffset']}
                clean_ents.append(cleaned)

            search_terms = []
            for e in clean_ents:
                #search_terms.append(e.get('name', ''))
                #search_terms.append(e.get('title', ''))
                # da capire se vogliamo che siano searchable, dato che surface contiene già il testo a cui è associata la entity
                search_terms.append(e.get('surface', ''))

            ent_text = " ".join(filter(None, search_terms)) # questa riga ha senso solo se prendiamo anche 'name' e 'title'
                                                                                           # se no ent_text va assegnato a e.get('surface', ' ')

            meta_json = json.dumps(clean_ents) # facciamo diventare tutti i metadati una stringa in forma json (non un oggetto dizionario, proprio una stringa)

            yield { # serve per lo stream dei dati quando viene chiamata createGenerator dentro indexer.index(•)
                "docno": str(row['para_id']),
                "text": row['context_ngrams'],
                "entities": ent_text, # entità searchable
                "entity_json": meta_json}
    # OCR
    if not context:
        for _, row in df.iterrows():
            # togliamo lOffset and rOffset
            clean_ents = []
            for ent in row['ner_entities_ocr']:
                cleaned = {k: v for k, v in ent.items() if k not in ['lOffset', 'rOffset']}
                clean_ents.append(cleaned)

            search_terms = []
            for e in clean_ents:
                #search_terms.append(e.get('name', ''))
                #search_terms.append(e.get('title', ''))
                # da capire se vogliamo che siano searchable, dato che surface contiene già il testo a cui è associata la entity
                search_terms.append(e.get('surface', ''))

            ent_text = " ".join(filter(None, search_terms)) # questa riga ha senso solo se prendiamo anche 'name' e 'title'
                                                                                           # se no ent_text va assegnato a e.get('surface', ' ')

            meta_json = json.dumps(clean_ents) # facciamo diventare tutti i metadati una stringa in forma json (non un oggetto dizionario, proprio una stringa)

            yield { # serve per lo stream dei dati quando viene chiamata createGenerator dentro indexer.index(•)
                "docno": str(row['para_id']),
                "text": row['raw_ocr_ngrams'],
                "entities": ent_text, # entità searchable
                "entity_json": meta_json}

contextIndex_path = 'data/docColl_context-index'
ocrIndex_path = 'data/docColl_ocr-index'

In [ ]:
if os.path.exists(contextIndex_path):
    shutil.rmtree(contextIndex_path)

indexerCont = pt.IterDictIndexer(
    'entity_index',
    fields=['text', 'entities'],
    meta={'docno', 'entity_json'})

indexrefCont = indexerCont.index(createGenerator(docColl_tok, context=True))

In [ ]:
if os.path.exists(ocrIndex_path):
    shutil.rmtree(ocrIndex_path)

indexerOCR = pt.IterDictIndexer(
    'entity_index',
    fields=['text', 'entities'],
    meta={'docno', 'entity_json'})

indexrefOCR = indexerOCR.index(createGenerator(docColl_tok, context=False))

#### Statistics about the indexed documents

In [ ]:
indexCont = pt.IndexFactory.of(indexrefCont)
stats = indexCont.getCollectionStatistics()
print('Index folder:', contextIndex_path)
print('Number of documents:', stats.getNumberOfDocuments())
print('Number of postings:', stats.getNumberOfPostings())
print('Number of tokens:', stats.getNumberOfTokens())
print('Number of unique terms:', stats.getNumberOfUniqueTerms())
print('Average document length:', stats.getAverageDocumentLength())

In [ ]:
indexOCR = pt.IndexFactory.of(indexrefOCR)
stats = indexOCR.getCollectionStatistics()
print('Index folder:', contextIndex_path)
print('Number of documents:', stats.getNumberOfDocuments())
print('Number of postings:', stats.getNumberOfPostings())
print('Number of tokens:', stats.getNumberOfTokens())
print('Number of unique terms:', stats.getNumberOfUniqueTerms())
print('Average document length:', stats.getAverageDocumentLength())

#### Query analysis

In [ ]:
display(queries.head(10))

--> da scrivere commento riguardo l'analisi delle queries

#### Qrels analysis

In [ ]:
display(qrels.sample(10))

In [ ]:
# stats for the qrels
# Count how many relevance assessments each query has
counts = qrels.groupby("query_id")["para_id"].count()  # group by query id and count documents
print('Overall Statistics')
print(counts.describe())  # show a summary of the count distribution

import matplotlib.pyplot as plt  # plotting library

# Plot how many relevance assessments each query received
plt.figure()  # create a new figure
counts.plot(kind='hist')  # histogram showing distribution of judgment counts
plt.xlabel('Number of relevance assessments per query')  # label for x-axis
plt.ylabel('Number of queries')  # label for y-axis
plt.title('Relevance assessment distribution')  # title of the plot
plt.show()  # display the plot

# Show the queries with the highest number of relevance assessments
counts.sort_values(ascending=False).head()  # top queries by number of judgments

# Count how many times each relevance label occurs overall
qrels['relevance'].value_counts()  # distribution of relevance scores (e.g., 0, 1, 2, etc.)

# Plot the label distribution as a histogram
plt.figure()  # create a new figure
qrels['relevance'].plot(kind='hist')  # histogram of relevance labels
plt.xlabel('Relevance score')  # label for x-axis
plt.ylabel('Frequency')  # label for y-axis
plt.title('Relevance score distribution')  # title of the plot
plt.show()  # display the plot

--> commento riguardo l'analisi delle qrels

## Phase I - Topical relevance-based retrieval

### **BM25 Retrieval from raw OCR (baseline 1)**

In [ ]:
bm25ocr = pt.terrier.Retriever(indexrefOCR, wmodel='BM25', ) # dovremmo usare un BM25F? per dividere i fields di ricerca (secondo me si)
res_bm25ocr = bm25ocr.transform()

### **BM25 Retrieval from corrected OCR (baseline 2)**

### **BM25 Retrieval from both raw and corrected OCR using RRF formula (baseline 3)**